#### Sql Alchemy and Python Execution Environment

In [1]:
import pandas as pd
import sqlite3

def load_data(database_loc, data_file,table_name):
    """Database located at best in the same location as the file.
    When the data is provided, ensure the full path is provided.
    The table with the given name will be created."""
    conn = sqlite3.connect(database_loc)
    dataframe = pd.read_csv(data_file)
    dataframe.drop('Unnamed: 0',axis=1,inplace=True)
    dataframe.to_sql(table_name, conn, if_exists='append',index=True)
    print(f'Data uploaded into {database_loc}')

In [2]:
db_loc = 'trial_data.db'
data_loc = '~/sample_data.csv'
table_name = 'hack_data'

#load_data(db_loc,data_loc,table_name)

In [3]:
from sqlalchemy import create_engine, select
from sqlalchemy.ext.declarative import declarative_base
#from sqlalchemy.orm import sessionmaker
from sqlalchemy.orm import Session
from sqlalchemy import Column, Integer, Float, String
from sqlalchemy.sql.sqltypes import TIMESTAMP

In [4]:
db_conn = create_engine('sqlite:///trial_data.db')
Base = declarative_base()

In [5]:
class Hacker(Base):
    """Hacker class provides access to the table hack_data in any Database"""
    __tablename__ = 'hack_data'
    Index = Column(Integer, primary_key=True,nullable=False)
    Datetime = Column(String)
    Host = Column(String)
    Src = Column(Integer)
    Proto = Column(String)
    Type = Column(String)
    Spt = Column(Float)
    Dpt = Column(Float)
    Srcstr = Column(String)
    Cc = Column(String)
    Country = Column(String)
    Locale = Column(String)
    Localeabbr = Column(String)
    Postalcode = Column(String)
    Latitude = Column(Float)
    Longitude = Column(Float)


In [6]:
with Session(db_conn) as connected:
    data_recd = connected.scalars(select(Hacker).limit(10))
    for row in data_recd: 
        print(row.Datetime)

7/9/13 6:08
8/26/13 23:41
8/26/13 23:59
8/24/13 15:41
7/4/13 13:05
5/31/13 15:59
8/11/13 8:38
9/2/13 13:35
5/30/13 5:02
7/2/13 20:33


https://github.com/tiangolo/pydantic-sqlalchemy

https://docs.sqlalchemy.org/en/14/orm/quickstart.html#declare-models

In [13]:
from pydantic_sqlalchemy import sqlalchemy_to_pydantic
PydanticHacker = sqlalchemy_to_pydantic(Hacker)

In [18]:
from sqlalchemy.orm import relationship, sessionmaker
HackerSession = sessionmaker(bind=db_conn)
db_exec : Session = HackerSession()

In [19]:
hackerData = db_exec.query(Hacker).first()

In [20]:
pyd_hacker_data = PydanticHacker.from_orm(hackerData)

In [21]:
pyd_hacker_data.dict()

{'Index': 0,
 'Datetime': '7/9/13 6:08',
 'Host': 'groucho-sa',
 'Src': 1017973657,
 'Proto': 'TCP',
 'Type': 'unknown',
 'Spt': 6000.0,
 'Dpt': 80.0,
 'Srcstr': '60.173.11.153',
 'Cc': 'CN',
 'Country': 'China',
 'Locale': 'Anhui Sheng',
 'Localeabbr': '34',
 'Postalcode': 'unknown',
 'Latitude': 31.8639,
 'Longitude': 117.2808}

In [33]:
class Tutorial(Base):
    """Tutorial class provides access to the table tutorial_pyd in any Database"""
    __tablename__ = 'tutorial_pyd'
    Index = Column(Integer, primary_key=True,nullable=False)
    Host = Column(String)
    Student = Column(String)
    Country = Column(String)
    Postalcode = Column(Integer)
    Latitude = Column(Float)
    Longitude = Column(Float)


In [35]:
Base.metadata.create_all(db_conn)

In [36]:
std1 = Tutorial(Host='Martian',Student='Venusian',
                Country='Tiberi',Postalcode=1578689,
               Latitude=576.68,Longitude=265.79)


In [39]:
db_exec.add(std1)
db_exec.commit()

In [40]:
tutorialData = db_exec.query(Tutorial).first()
pydanticTutorial = sqlalchemy_to_pydantic(Tutorial)
tutorialOutput = pydanticTutorial.from_orm(tutorialData)
tutorialOutput

Tutorial(Index=1, Host='Martian', Student='Venusian', Country='Tiberi', Postalcode=1578689, Latitude=576.68, Longitude=265.79)